# **IBM Applied Data Science Capstone Course by Coursera**
**_Opening a New Cafe in Jakarta, Indonesia_**

### **1. Import libraries**

In [1]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 102kB 3.6MB/s 


In [2]:
import folium
import geocoder
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

### **2. Scrap data from Wikipedia page into a DataFrame**

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Central_Jakarta").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = [item2.text for item in soup.find_all('td', class_='navbox-list') for item2 in item.find_all('a')]

In [6]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()

,Neighborhood
0,Cempaka Putih
1,Gambir
2,Johar Baru
3,Kemayoran
4,Menteng


In [7]:
df.shape

(44, 1)

### **3. Get the geographical coordinates**

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
  # initialize your variable to None
  lat_lng_coords = None
  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Jakarta, Indonesia'.format(neighborhood))
    lat_lng_coords = g.latlng
  return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [20]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [21]:
# check the neighborhoods and the coordinates
print(df.shape)
df.head()

(44, 3)


,Neighborhood,Latitude,Longitude
0,Cempaka Putih,-6.17600,106.87061
1,Gambir,-6.17299,106.81571
2,Johar Baru,-6.18588,106.85598
3,Kemayoran,-6.16584,106.85396
4,Menteng,-6.20311,106.83798


In [22]:
# save the DataFrame as CSV file
df.to_csv("df.csv", index=False)

### **4. Create a map of Kuala Lumpur with neighborhoods superimposed on top**

In [23]:
# get the coordinates of Jakarta
address = 'Jakarta, Indonesia'

geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Jakarta, Indonesia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Jakarta, Indonesia -6.1753942, 106.827183.


In [25]:
# create map of Jakarta using latitude and longitude values
map_jkt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_jkt)  
    
map_jkt

### **5. Use the Foursquare API to explore the neighborhoods**

In [26]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [27]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3383, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cempaka Putih,-6.176,106.87061,Pagi Sore,-6.176376,106.871474,Padangnese Restaurant
1,Cempaka Putih,-6.176,106.87061,DIMSUM-ING,-6.176087,106.872188,Dim Sum Restaurant
2,Cempaka Putih,-6.176,106.87061,Surabi Enhaii Cempaka Putih,-6.176321,106.872056,Café
3,Cempaka Putih,-6.176,106.87061,Soto Jakarta - Mas Hari,-6.172202,106.874084,Soup Place
4,Cempaka Putih,-6.176,106.87061,Starbucks,-6.177065,106.875270,Coffee Shop


**Let's check how many venues were returned for each neighorhood**

In [29]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Cakung,27,27,27,27,27,27
Cempaka Putih,86,86,86,86,86,86
Cengkareng,67,67,67,67,67,67
Cilandak,100,100,100,100,100,100
Cilincing,3,3,3,3,3,3
Cipayung,19,19,19,19,19,19
Ciracas,33,33,33,33,33,33
Duren Sawit,76,76,76,76,76,76
Gambir,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 255 uniques categories.


In [31]:
# print out the list of categories
_VenueCategory = venues_df['VenueCategory'].unique()
_VenueCategory.sort()
_VenueCategory

array(['Accessories Store', 'Acehnese Restaurant', 'African Restaurant',
       'Airport', 'American Restaurant', 'Aquarium', 'Arcade',
       'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'Auto Dealership',
       'Automotive Shop', 'BBQ Joint', 'Badminton Court', 'Bakery',
       'Balinese Restaurant', 'Bank', 'Bar', 'Baseball Stadium',
       'Basketball Court', 'Beach', 'Bed & Breakfast', 'Beer Bar',
       'Beer Garden', 'Betawinese Restaurant', 'Big Box Store', 'Bistro',
       'Board Shop', 'Boarding House', 'Boat or Ferry', 'Bookstore',
       'Boutique', 'Bowling Alley', 'Brazilian Restaurant',
       'Breakfast Spot', 'Bridal Shop', 'Bubble Tea Shop', 'Buffet',
       'Building', 'Burger Joint', 'Cable Car', 'Cafeteria', 'Café',
       'Camera Store', 'Campground', 'Cantonese Restaurant',
       'Cheese Shop', 'Chinese Restaurant', 'Church', 'Clothing Store',
       'Coffee Shop', 'College Academic Building', 'College Caf

### **6. Analyze Each Neighborhood**

In [32]:
# one hot encoding
jkt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jkt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jkt_onehot.columns[-1]] + list(jkt_onehot.columns[:-1])
jkt_onehot = jkt_onehot[fixed_columns]

print(jkt_onehot.shape)
jkt_onehot.head()

(3383, 256)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,...,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Travel Agency,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Women's Store,Zoo
0,Cempaka Putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cempaka Putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cempaka Putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cempaka Putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Cempaka Putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [33]:
jkt_grouped = jkt_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(jkt_grouped.shape)
jkt_grouped.head()

(44, 256)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,...,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Travel Agency,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wings Joint,Women's Store,Zoo
0,Cakung,0.0,0.000000,0.0,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.037037,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
1,Cempaka Putih,0.0,0.011628,0.0,0.000000,0.0,0.0,0.023256,0.0,0.0,0.0,0.023256,0.011628,0.0,0.0,0.011628,0.0,0.011628,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.011628,0.0,0.0,0.0,0.011628,0.0,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.000000,0.0,0.011628,0.0,0.00,0.0,0.011628,0.0,0.0,0.011628,0.0,0.0,0.0,0.00,0.0,0.011628,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.011628,0.0,0.0
2,Cengkareng,0.0,0.000000,0.0,0.014925,0.0,0.0,0.000000,0.0,0.0,0.0,0.104478,0.000000,0.0,0.0,0.014925,0.0,0.014925,0.0,0.029851,0.014925,0.0,0.0,0.0,0.0,0.014925,0.0,0.0,0.0,0.029851,0.0,0.0,0.0,0.014925,0.0,0.0,0.0,0.00,0.0,0.014925,...,0.00,0.0,0.0,0.014925,0.0,0.014925,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014925,0.00,0.000000,0.0,0.0
3,Cilandak,0.0,0.010000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.040000,0.000000,0.0,0.0,0.010000,0.0,0.020000,0.0,0.000000,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,...,0.02,0.0,0.0,0.000000,0.0,0.040000,0.0,0.01,0.0,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.000000,0.0,0.0
4,Cilincing,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0


In [34]:
len(jkt_grouped[jkt_grouped["Café"] > 0])

33

**Create a new DataFrame for Cafe venues data only**

In [37]:
jkt_cafe = jkt_grouped[["Neighborhoods","Café"]]

In [38]:
jkt_cafe.head()

,Neighborhoods,Café
0,Cakung,0.000000
1,Cempaka Putih,0.046512
2,Cengkareng,0.029851
3,Cilandak,0.060000
4,Cilincing,0.000000


### **7. Cluster Neighborhoods**
**Run k-means to cluster the neighborhoods in Jakarta into 3 clusters.**

In [39]:
# set number of clusters
kclusters = 3

jkt_clustering = jkt_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jkt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1, 0, 0, 0, 0, 0, 2], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
jkt_merged = jkt_mall.copy()

# add clustering labels
jkt_merged["Cluster Labels"] = kmeans.labels_

In [41]:
jkt_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
jkt_merged.head()

,Neighborhood,Café,Cluster Labels
0,Cakung,0.000000,0
1,Cempaka Putih,0.046512,1
2,Cengkareng,0.029851,2
3,Cilandak,0.060000,1
4,Cilincing,0.000000,0


In [43]:
# merge jakarta_grouped with jakarta_data to add latitude/longitude for each neighborhood
jkt_merged = jkt_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(jkt_merged.shape)
jkt_merged.head() # check the last columns!

(44, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.000000,0,-6.19623,106.93522
1,Cempaka Putih,0.046512,1,-6.17600,106.87061
2,Cengkareng,0.029851,2,-6.13060,106.74559
3,Cilandak,0.060000,1,-6.29051,106.79491
4,Cilincing,0.000000,0,-6.11358,106.94911


In [44]:
# sort the results by Cluster Labels
print(jkt_merged.shape)
jkt_merged.sort_values(["Cluster Labels"], inplace=True)
jkt_merged.head()

(44, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.00,0,-6.19623,106.93522
41,Tanah Abang,0.01,0,-6.20785,106.81082
40,Tambora,0.01,0,-6.15194,106.80777
39,Taman Sari,0.00,0,-6.15327,106.82357
38,South Thousand Islands,0.00,0,-5.60000,106.55000


**Finally, let's visualize the resulting clusters**

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jkt_merged['Latitude'], jkt_merged['Longitude'], 
                                  jkt_merged['Neighborhood'], jkt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### **8. Examine Clusters**

#### **Cluster 0**

In [47]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.000000,0,-6.19623,106.93522
41,Tanah Abang,0.010000,0,-6.20785,106.81082
40,Tambora,0.010000,0,-6.15194,106.80777
39,Taman Sari,0.000000,0,-6.15327,106.82357
38,South Thousand Islands,0.000000,0,-5.60000,106.55000
35,Sawah Besar,0.010000,0,-6.15741,106.83140
31,Pasar Rebo,0.000000,0,-6.33769,106.85755
29,Pancoran,0.010000,0,-6.25548,106.84391
28,Palmerah,0.010000,0,-6.19153,106.79556
26,North Thousand Islands,0.000000,0,-5.60000,106.55000


#### **Cluster 1**

In [48]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
36,Senen,0.050000,1,-6.19675,106.84848
25,Menteng,0.060000,1,-6.20311,106.83798
3,Cilandak,0.060000,1,-6.29051,106.79491
11,Jatinegara,0.063291,1,-6.23265,106.88539
42,Tanjung Priok,0.050000,1,-6.12842,106.85952
13,Kalideres,0.081633,1,-6.12210,106.70727
1,Cempaka Putih,0.046512,1,-6.17600,106.87061
32,Penjaringan,0.050000,1,-6.11662,106.77803
33,Pesanggrahan,0.049180,1,-6.23640,106.75447


#### **Cluster 2**

In [49]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
37,Setiabudi,0.020000,2,-6.22124,106.82528
2,Cengkareng,0.029851,2,-6.13060,106.74559
16,Kebon Jeruk,0.030000,2,-6.19706,106.77303
30,Pasar Minggu,0.030000,2,-6.29265,106.82791
15,Kebayoran Lama,0.030000,2,-6.26386,106.77519
27,Pademangan,0.020000,2,-6.13439,106.83661
9,Grogol Petamburan,0.020000,2,-6.16777,106.78460
10,Jagakarsa,0.034483,2,-6.34308,106.81745
20,Koja,0.025000,2,-6.12716,106.89919
17,Kelapa Gading,0.030000,2,-6.16021,106.90478
